In [1]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWhdoxBt2zjtNGpTE8MqKQs65ujz0VXx9FPDYa8Pu-OSyT4wVNq0ezY

Successfully saved authorization token.


In [3]:
!pip install altair

In [4]:
import pandas as pd
import altair as alt
import numpy as np
import folium

In [5]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline

In [6]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [7]:
# creates region of interest by producing coordinates
geoJSON ={
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              33.20068359375,
              -14.072644954380316
            ],
            [
              30.377197265625,
              -15.294782590260931
            ],
            [
              33.013916015625,
              -16.709862933206566
            ],
            [
              34.43115234375,
              -15.262988555023204
            ],
            [
              33.20068359375,
              -14.072644954380316
            ]
          ]
        ]
      }
    }
  ]
}

In [8]:
#redion of interest
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)


This tutorial provides methods for generating time series data in Earth Engine and visualizing it with the Altair library using  vegetation response as an example

In [9]:
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=1000,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
  """Creates a region reduction function.

  Creates a region reduction function intended to be used as the input function
  to ee.ImageCollection.map() for reducing pixels intersecting a provided region
  to a statistic for each image in a collection. See ee.Image.reduceRegion()
  documentation for more details.

  Args:
    geometry:
      An ee.Geometry that defines the region over which to reduce data.
    reducer:
      Optional; An ee.Reducer that defines the reduction method.
    scale:
      Optional; A number that defines the nominal scale in meters of the
      projection to work in.
    crs:
      Optional; An ee.Projection or EPSG string ('EPSG:5070') that defines
      the projection to work in.
    bestEffort:
      Optional; A Boolean indicator for whether to use a larger scale if the
      geometry contains too many pixels at the given scale for the operation
      to succeed.
    maxPixels:
      Optional; A number specifying the maximum number of pixels to reduce.
    tileScale:
      Optional; A number representing the scaling factor used to reduce
      aggregation tile size; using a larger tileScale (e.g. 2 or 4) may enable
      computations that run out of memory with the default.

  Returns:
    A function that accepts an ee.Image and reduces it by region, according to
    the provided arguments.
  """

  def reduce_region_function(img):
    """Applies the ee.Image.reduceRegion() method.

    Args:
      img:
        An ee.Image to reduce to a statistic by region.

    Returns:
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """

    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()})
  return reduce_region_function

In [10]:
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

Load the Landsat8, ee.FeatureCollection and filter it to include only the Mozambique, which defines the area of interest (AOI).

### Import and reduce

1. Load the Landast-8 NDVI data as an `ee.ImageCollection`.
1. Create a region reduction function.
3. Apply the function to all images in the time series.
4. Filter out features with null computed values.

In [11]:

ndvi = ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_NDVI').filterDate('2019-01-01', '2019-12-31').select('NDVI')

reduce_ndvi = create_reduce_region_function(
    geometry=aoi, reducer=ee.Reducer.mean(), scale=1000, crs='EPSG:3310')

ndvi_stat_fc = ee.FeatureCollection(ndvi.map(reduce_ndvi)).filter(
    ee.Filter.notNull(ndvi.first().bandNames()))

### Prepare DataFrame

1. Transfer data from the server to the client.
2. Convert the Python dictionary to a pandas DataFrame.
3. Preview the DataFrame and check data types.

In [12]:
ndvi_dict = fc_to_dict(ndvi_stat_fc).getInfo()
ndvi_df = pd.DataFrame(ndvi_dict)
display(ndvi_df)
print(ndvi_df.dtypes)

,NDVI,millis,system:index
0,0.373672,1546300800000,20190101
1,0.366012,1546992000000,20190109
2,0.087306,1547683200000,20190117
3,0.446114,1548374400000,20190125
4,0.226723,1549065600000,20190202
5,0.487012,1549756800000,20190210
6,0.651029,1550448000000,20190218
7,0.497158,1551139200000,20190226
8,0.468859,1551830400000,20190306
9,0.404626,1552521600000,20190314


NDVI            float64
millis            int64
system:index     object
dtype: object


In [13]:
# Function to add date variables to DataFrame.
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df


Add date attribute columns.

Preview the DataFrame.

In [14]:
#ndvi_df['NDVI'] = ndvi_df['NDVI']
ndvi_df = add_date_info(ndvi_df)
ndvi_df.head(5)

,NDVI,millis,system:index,Timestamp,Year,Month,Day,DOY
0,0.373672,1546300800000,20190101,2019-01-01,2019,1,1,1
1,0.366012,1546992000000,20190109,2019-01-09,2019,1,9,9
2,0.087306,1547683200000,20190117,2019-01-17,2019,1,17,17
3,0.446114,1548374400000,20190125,2019-01-25,2019,1,25,25
4,0.226723,1549065600000,20190202,2019-02-02,2019,2,2,33


In [15]:
ndvi_df['NDVI'].min()

0.08730591366869442

In [16]:

ndvi_df['NDVI'].max()


0.651029354843719

### DAY line chart

Make a day of Months line chart where each line represents a month of observations. This chart makes it possible to compare the same observation date among months. Use it to compare NDVI values for years during the drought and not.

Day of month is represented on the x-axis and NDVI on the y-axis. Each line represents a month and is distinguished by color. Note that this plot includes a tooltip and has been made interactive so that the axes can be zoomed and panned.

In [21]:
highlight = alt.selection(
    type='single', on='mouseover', fields=['Month'], nearest=True)

base = alt.Chart(ndvi_df).encode(
    x=alt.X('Day:Q', scale=alt.Scale(domain=[0, 31], clamp=True)),
    y=alt.Y('NDVI:Q', scale=alt.Scale(domain=[-1, 1])),
    color=alt.Color('Month:O', scale=alt.Scale(scheme='magma')))

points = base.mark_circle().encode(
    opacity=alt.value(0),
    tooltip=[
        alt.Tooltip('Month:O', title='Month'),
        alt.Tooltip('Day:Q', title='Day'),
        alt.Tooltip('NDVI:Q', title='NDVI')
    ]).add_selection(highlight)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3)))

(points + lines).properties(width=600, height=350).interactive()

alt.LayerChart(...)

Another way to view these data is to plot the distribution of NDVI by DaY represented as an interquartile range envelope and median line. Here, these two charts are defined and then combined in the following snippet.

1. Define a base chart.
2. Define a line chart for median NDVI (note the use of aggregate median transform grouping by DaY).
3. Define a band chart using `'iqr'` (interquartile range) to represent NDVI distribution grouping on DaY.
4. Combine the line and band charts.

In [18]:
ndvi_df['NDVI'].median()

0.2994992496573225

In [19]:
base = alt.Chart(ndvi_df).encode(
    x=alt.X('Day:Q', scale=alt.Scale(domain=(0, 31))))

line = base.mark_line().encode(
    y=alt.Y('median(NDVI):Q', scale=alt.Scale(domain=(-1, 1))))

band = base.mark_errorband(extent='iqr').encode(
    y='NDVI:Q')

(line + band).properties(width=600, height=300).interactive()

alt.LayerChart(...)